In [1]:
library(pogit)
# PIP, , what exactly do these 2 weights mean
# how to deal with the intercept: did not make difference if centering

In [2]:
block = c(79,137)
block_simu = c(1215, 1273)
# block = c(23, 36)
# block_simu = c(164, 177)

In [3]:
X_ori <- as.matrix(data.table::fread(paste("/data/GIT/cnv-gene-mapping/data/deletion_simu/block_", block[1], "_" , block[2], 
                                           "/deletion.genes.block30.for_simu.sample.genes.block_", block[1], "_" , block[2], ".gz", sep = ""), header = F))

In [4]:
X_center <- scale(X_ori, scale = FALSE)

In [5]:
head(X_center)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05
-0.005592007,-0.0005219207,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,-0.000820161,⋯,-0.003280644,-0.002311363,-0.002311363,-0.0005964808,-0.0005964808,-0.0005964808,-0.0003728005,-0.0003728005,-7.45601e-05,-7.45601e-05


In [6]:
dim(X_ori)

[1] 13412    59

In [7]:
y <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.y"))

In [8]:
length(y)

[1] 13412

In [9]:
# beta <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.beta"))

In [10]:
# mean(beta); sd(beta); sum(beta == 0); sum(beta == 0)/length(beta)

In [11]:
X_intercept <- cbind(rep(1, nrow(X_ori)), X_ori)

In [12]:
y <- as.numeric(y[,1])

In [13]:
N <- rep(1, length(y))

In [14]:
prior <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.updateb0.rds")

In [15]:
mu0 = sum(prior$b0 * prior$w)

In [16]:
mu0

[1] 0.7770721

In [17]:
var = sum(prior$sa * prior$w)

In [18]:
var

[1] 0.7117456

In [19]:
pi0 = sum((10^(prior$logodds) / (1 + 10^(prior$logodds))) * prior$w)

In [20]:
pi0

[1] 0.0437755

In [21]:
# b = prior$beta[which(prior$beta != 0)]
# mean(b); var(b); sum(prior$beta == 0); sum(prior$beta == 0)/length(prior$beta)

In [22]:
X <- X_center

In [23]:
res <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X_ori)), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, pb0 = 10)), 
                mcmc = list(M = 5000, burnin = 1000, msave = TRUE), BVS = FALSE)


MCMC for the binomial logit model:

it = 1 /--- duration of MCMC so far: 26.2 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 26.32 sec.,  expected time to end: 2631.86  min. 
it = 3 /--- duration of MCMC so far: 26.44 sec.,  expected time to end: 1321.81  min. 
it = 4 /--- duration of MCMC so far: 26.57 sec.,  expected time to end: 885.32  min. 
it = 5 /--- duration of MCMC so far: 26.7 sec.,  expected time to end: 667.03  min. 
it = 10 /--- duration of MCMC so far: 27.46 sec.,  expected time to end: 304.63  min. 
it = 20 /--- duration of MCMC so far: 29.18 sec.,  expected time to end: 153.09  min. 
it = 50 /--- duration of MCMC so far: 32.88 sec.,  expected time to end: 66.56  min. 
it = 100 /--- duration of MCMC so far: 39.11 sec.,  expected time to end: 38.86  min. 
it = 200 /--- duration of MCMC so far: 50.34 sec.,  expected time to end: 24.46  min. 
it = 500 /--- duration of MCMC so far: 83.28 sec.,  expected time to end: 15.3  min. 
it = 1000 /--- d

In [26]:
saveRDS(res, paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".pi.rds", sep = ""))

In [24]:
res1 <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X_ori)), V = var, w = c(wa0 = 1.412, wb0 = 10), pi = c(pa0 = 1, pb0 = 1)), 
                 mcmc = list(M = 5000, burnin = 1000, msave = TRUE), BVS = FALSE)


MCMC for the binomial logit model:

it = 1 /--- duration of MCMC so far: 25.29 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 25.39 sec.,  expected time to end: 2538.18  min. 
it = 3 /--- duration of MCMC so far: 25.49 sec.,  expected time to end: 1274.13  min. 
it = 4 /--- duration of MCMC so far: 25.6 sec.,  expected time to end: 852.91  min. 
it = 5 /--- duration of MCMC so far: 25.71 sec.,  expected time to end: 642.3  min. 
it = 10 /--- duration of MCMC so far: 26.22 sec.,  expected time to end: 290.89  min. 
it = 20 /--- duration of MCMC so far: 27.32 sec.,  expected time to end: 143.31  min. 
it = 50 /--- duration of MCMC so far: 30.48 sec.,  expected time to end: 61.7  min. 
it = 100 /--- duration of MCMC so far: 35.73 sec.,  expected time to end: 35.5  min. 
it = 200 /--- duration of MCMC so far: 46.2 sec.,  expected time to end: 22.44  min. 
it = 500 /--- duration of MCMC so far: 77.58 sec.,  expected time to end: 14.25  min. 
it = 1000 /--- dur

In [29]:
saveRDS(res1, paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".omega.rds", sep = ""))

In [27]:
summary(res)

MCMC for the logit model:

Call:
logitBvs(y = y, N = N, X = X, prior = list(slab = "Normal", aj0 = rep(mu0, 
    ncol(X_ori)), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, 
    pb0 = 10)), mcmc = list(M = 5000, burnin = 1000, msave = TRUE), 
    BVS = FALSE)


MCMC:
M = 5000 draws after a burn-in of 1000
Thinning parameter: 1

Prior: Normal prior [V=0.711745609189383]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
 0.000  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[11] a0[12] a0[13] a0[14] a0[15] a0[16] a0[17] a0[18] a0[19] a0[20] a0[21] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[22] a0[23] a0[24] a0[25] a0[26] a0[27] a0[28] a0[29] a0[30] a0[31] a0[32] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[33] a0[34] a0[35] a0[36] a0[37] a0[38] a0[39] a0[40] a0[41] a0[42] a0[43] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.

In [32]:
names(res1)

[1] "samplesL"    "data"        "model.logit" "mcmc"        "prior.logit"
 [6] "dur"         "BVS"         "start"       "family"      "call"

In [53]:
names(res1$samplesL)

[1] "alpha"       "pdeltaAlpha" "psiAlpha"    "thetaAlpha"  "pgammaAlpha"
[6] "ai"

In [119]:
sum(res1$samplesL$pdeltaAlpha)

[1] 354000

In [196]:
aj0_tmp = c(rep(0, 29), 0, rep(0, 29))

In [202]:
res_mu0 <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = aj0_tmp, V = var, w = c(wa0 = 1.412, wb0 = 10), pi = c(pa0 = 1, pb0 = 1)), 
                    mcmc = list(M = 6, burnin = 2, startsel = 1, msave = FALSE), BVS = TRUE)


MCMC for the binomial logit model with variable selection:

it = 1 /--- duration of MCMC so far: 24.82 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 24.93 sec.,  expected time to end: 2.91  min. 
it = 3 /--- duration of MCMC so far: 30.58 sec.,  expected time to end: 1.53  min. 
it = 4 /--- duration of MCMC so far: 35.38 sec.,  expected time to end: 0.98  min. 
it = 5 /--- duration of MCMC so far: 39.37 sec.,  expected time to end: 0.66  min. 
it = 8 /--- duration of MCMC (total): 44.39 sec. 
 


In [203]:
names(res_mu0$samplesL)

[1] "alpha"       "pdeltaAlpha" "psiAlpha"    "thetaAlpha"  "pgammaAlpha"
[6] "ai"

In [204]:
Filter(Negate(is.null), res_mu0$samplesL)

alpha.0,alpha.1,alpha.2,alpha.3,alpha.4,alpha.5,alpha.6,alpha.7,alpha.8,alpha.9,⋯,alpha.50,alpha.51,alpha.52,alpha.53,alpha.54,alpha.55,alpha.56,alpha.57,alpha.58,alpha.59
-0.0252518542,-1.7665745,1.6120074,-0.3948854,0.4565698,0.2425785,-0.8092669,-1.0030502,2.3954637,0.4602667,⋯,0.5335080,0.01634485,0.5814829,0.4946527,-1.34048820,1.1488151,-0.22614725,0.2293970,-0.1564463,0.2864377
0.0033881734,-0.8420072,0.4435998,-2.8966846,-0.1940203,0.1539358,-0.2427303,-0.9475327,-0.5695298,0.0000000,⋯,0.2605585,0.68082006,-1.1227716,0.0000000,0.59749111,0.1807795,-0.28951579,-0.7903163,-0.8129027,0.5611303
0.0399952188,0.6378380,-0.3209207,0.0000000,0.0000000,-0.5919549,0.2091963,-0.1734753,-0.2217359,0.5463862,⋯,-0.4453246,-1.20630500,0.0000000,-0.7083775,0.01688112,0.0000000,0.03370242,1.2561097,-0.9307225,-1.0977128
0.0115263602,-0.6055566,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.0000000,-0.08834737,-0.9068963,0.0000000,0.00000000,0.0000000,0.00000000,-0.4516599,-2.2563741,-1.4160819
0.0048777583,0.0000000,0.0000000,0.0000000,0.0000000,-0.2365425,0.0000000,0.0000000,0.0000000,0.0000000,⋯,0.4170449,0.00000000,-0.8719510,0.0000000,0.31577137,0.0000000,-0.62599621,0.0000000,0.0000000,0.0000000
0.0037599214,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,-0.2009508,⋯,0.0000000,-1.66274258,0.0000000,0.0000000,0.00000000,0.0000000,-0.63268368,1.3065045,-1.2901685,0.0000000
-0.0078225405,0.0000000,-0.3196419,0.0000000,0.0000000,0.6915492,0.0000000,0.2305308,0.0000000,1.1854659,⋯,0.0000000,-1.24505488,0.0000000,0.0000000,0.00000000,0.0000000,0.00000000,0.0000000,0.3578740,-1.6534579
0.0009612315,0.0000000,0.0000000,0.0000000,-0.1486452,0.0000000,0.0000000,0.5489134,-1.0797960,0.0000000,⋯,0.0000000,0.00000000,0.0000000,-1.4560184,0.00000000,0.0000000,-0.05249286,0.0000000,0.0000000,0.0000000
pdeltaA.1,pdeltaA.2,pdeltaA.3,pdeltaA.4,pdeltaA.5,pdeltaA.6,pdeltaA.7,pdeltaA.8,pdeltaA.9,pdeltaA.10,⋯,pdeltaA.50,pdeltaA.51,pdeltaA.52,pdeltaA.53,pdeltaA.54,pdeltaA.55,pdeltaA.56,pdeltaA.57,pdeltaA.58,pdeltaA.59
1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000


In [200]:
mean(res_mu0$samplesL$pdeltaAlpha[3:8,2])

[1] 0.714597

In [201]:
summary(res_mu0)

Bayesian variable selection for the logit model:

Call:
logitBvs(y = y, N = N, X = X, prior = list(slab = "Normal", aj0 = aj0_tmp, 
    V = var, w = c(wa0 = 1.412, wb0 = 10), pi = c(pa0 = 1, pb0 = 1)), 
    mcmc = list(M = 6, burnin = 2, startsel = 1, msave = TRUE), 
    BVS = TRUE)


MCMC:
M = 6 draws after a burn-in of 2
BVS started after 1 iterations
Thinning parameter: 1

Prior:  spike-and-slab prior with Normal slab [V=0.711745609189383]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
     0      0      0      0      0      0      0      0      0      0      0 
a0[11] a0[12] a0[13] a0[14] a0[15] a0[16] a0[17] a0[18] a0[19] a0[20] a0[21] 
     0      0      0      0      0      0      0      0      0      0      0 
a0[22] a0[23] a0[24] a0[25] a0[26] a0[27] a0[28] a0[29] a0[30] a0[31] a0[32] 
     0      0      0      0      0      0      0      0      0      0      0 
a0[33] a0[34] a0[35] a0[36] a0[37] a0[38] a0[39] a0[40] a0[41] a0[42] a0[43] 
     0

In [51]:
tmp <- res1

In [52]:
names(tmp)

[1] "samplesL"    "data"        "model.logit" "mcmc"        "prior.logit"
 [6] "dur"         "BVS"         "start"       "family"      "call"

In [53]:
names(tmp$samplesL)

[1] "alpha"       "pdeltaAlpha" "psiAlpha"    "thetaAlpha"  "pgammaAlpha"
[6] "ai"

In [55]:
bhat <- colMeans(tmp$samplesL$alpha[,-1])
bhat1 <- colMeans(tmp$samplesL$alpha)

In [56]:
# yhat <- exp(X %*% bhat)/(1+exp(X %*% bhat))
yhat <- X_ori %*% bhat
yhat1 <- X_center %*% bhat
yhat2 <- X_intercept %*% bhat1

In [57]:
min(yhat); max(yhat); mean(yhat); median(yhat)

[1] -1.079004

[1] 1.650473

[1] 0.001295595

[1] 0

In [58]:
b_all <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.beta"))
b <- b_all[(block_simu[1]+1):(block_simu[2]+1)]

In [59]:
# y <- exp(X %*% b)/(1+exp(X %*% b))
yb <- X_ori %*% b

In [60]:
min(yb); max(yb); mean(yb); median(yb)

[1] 0

[1] 0.9578892

[1] 0.004438093

[1] 0

In [63]:
cor(yb, yhat, method = "pearson")

0.3444027


In [39]:
min(yhat1[yhat1<0])

[1] -0.111294

In [40]:
cor.test(yb, yhat, method=c("pearson"))


	Pearson's product-moment correlation

data:  yb and yhat
t = 874.81, df = 13410, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.9910558 0.9916388
sample estimates:
      cor 
0.9913522 


In [26]:
summary(tmp)

Bayesian variable selection for the logit model:

Call:
logitBvs(y = y, N = N, X = X)


MCMC:
M = 8000 draws after a burn-in of 2000
BVS started after 1000 iterations
Thinning parameter: 1

Prior:  spike-and-slab prior with Student-t slab [V=5]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
     0      0      0      0      0      0      0      0      0      0      0 
a0[11] a0[12] a0[13] a0[14] 
     0      0      0      0 
 w[a]  w[b] pi[a] pi[b] 
    1     1     1     1 


Model averaged posterior means, estimated posterior inclusion
 probabilities and 95%-HPD intervals:

            Estimate P(.=1) 95%-HPD[l] 95%-HPD[u]
(Intercept)    0.029     NA     -0.012      0.067
alpha.1        0.216  0.512     -0.099      0.916
alpha.2        0.426  0.543     -2.258      3.523
alpha.3        0.458  0.541     -2.258      3.663
alpha.4        0.448  0.541     -2.249      3.660
alpha.5        0.446  0.542     -2.097      3.782
alpha.6        0.449  0.544     -2.23

In [92]:
pdf(file = paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".var.selection.beta.hist.pdf", sep = ""))
par(mfrow = c(2,2))
for (i in seq(2, (block[2]-block[1]+2))){
    hist(res$samplesL$alpha[,i], breaks = 80, main = paste("beta", i-1, "histogram"), xlab = paste("beta", i-1))
}
dev.off()

png 
  2

In [94]:
dim(res$samplesL$alpha)

[1] 12000    15

In [79]:
# mean(res$samplesL$alpha[,42])

In [78]:
summary(res)

MCMC for the logit model:

Call:
logitBvs(y = y, N = N, X = X, prior = list(slab = "Normal", aj0 = rep(mu0, 
    ncol(X) - 1), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, 
    pb0 = 10)), mcmc = list(M = 24000, burnin = 6000, msave = TRUE), 
    BVS = FALSE)


MCMC:
M = 24000 draws after a burn-in of 6000
Thinning parameter: 1

Prior: Normal prior [V=0.711745609189383]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
 0.000  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[11] a0[12] a0[13] a0[14] a0[15] a0[16] a0[17] a0[18] a0[19] a0[20] a0[21] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[22] a0[23] a0[24] a0[25] a0[26] a0[27] a0[28] a0[29] a0[30] a0[31] a0[32] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[33] a0[34] a0[35] a0[36] a0[37] a0[38] a0[39] a0[40] a0[41] a0[42] a0[43] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  

In [70]:
data(simul_binomial)

In [71]:
y <- simul_binomial$y
N <- simul_binomial$N
X <- as.matrix(simul_binomial[, -c(1,2)])

In [72]:
length(y); length(N); dim(X)

[1] 512

[1] 512

[1] 512  10

In [73]:
head(X)

X.0,X.1,X.2,X.3,X.4,X.5,X.6,X.7,X.8,X.9
1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,1


In [74]:
prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X_ori)), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, pb0 = 10))

[1] 512

In [87]:
m3 <- logitBvs(y = y, N = N, X = X, mcmc = list(M = 4, burnin = 1), prior = list(slab = "Normal", aj0 = rep(0, ncol(X)-1)), 
               model = list(deltafix = c(1, 1, 1, 0, 0, 0, 1, 0, 0)))

Warning message:
“invalid 'BVS' or 'startsel' argument:
'startsel' was set to 0.5”


MCMC for the binomial logit model with variable selection:

it = 1 /--- duration of MCMC so far: 1.1 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 1.11 sec.,  expected time to end: 0.07  min. 
it = 3 /--- duration of MCMC so far: 1.13 sec.,  expected time to end: 0.03  min. 
it = 4 /--- duration of MCMC so far: 1.15 sec.,  expected time to end: 0.01  min. 
it = 5 /--- duration of MCMC so far: 1.17 sec.,  expected time to end: 0  min. 


In [88]:
summary(m3)

Bayesian variable selection for the binomial logit model:

Call:
logitBvs(y = y, N = N, X = X, model = list(deltafix = c(1, 1, 
    1, 0, 0, 0, 1, 0, 0)), prior = list(slab = "Normal", aj0 = rep(0, 
    ncol(X) - 1)), mcmc = list(M = 4, burnin = 1))


MCMC:
M = 4 draws after a burn-in of 1
BVS started after 0.5 iterations
Thinning parameter: 1

Prior:  spike-and-slab prior with Normal slab [V=5]

a0[0] a0[1] a0[2] a0[3] a0[4] a0[5] a0[6] a0[7] a0[8] a0[9] 
    0     0     0     0     0     0     0     0     0     0 
 w[a]  w[b] pi[a] pi[b] 
    1     1     1     1 


Model averaged posterior means, estimated posterior inclusion
 probabilities and 95%-HPD intervals:

            Estimate P(.=1) 95%-HPD[l] 95%-HPD[u]
(Intercept)   -0.419     NA     -0.483     -0.363
alpha.1        0.156  1.000      0.130      0.190
alpha.2       -0.106  1.000     -0.124     -0.075
alpha.3        0.072  1.000      0.019      0.127
alpha.4       -0.874  1.000     -0.946     -0.726
alpha.5        0.000  0.0